# Bibliotecas Usadas

In [1]:
import numpy as np
import bisect 
import time
import random
import copy

# O Jogo

Abaixo, está a classe elaborada para simular o jogo slide number, necessário para fazer as possibilidades de partida 

In [2]:
class Jogo:
    
    """
    Para o construtor, é necessário informar a dimensão, do tabuleiro quadrado, e uma jogada anterior
    que, por default, não existe. O construtor faz um tabuleiro gabarito
    """
    def __init__(self, dimensao, jogada_anterior=None):
        
        tabuleiro = []
        peso = {}
        coordinate = {}
        k=1
        
        #loop tradicional para fazer uma matriz
        for i in range(0,dimensao):
            tabuleiro.append([])
            for j in range(0, dimensao):
                tabuleiro[i].append(k)
                peso[k] = 0
                coordinate[k] = (i, j)
                k+=1
        
        #Espaço vazio do jogo                        
        tabuleiro[-1][-1] = None
        peso.pop(dimensao**2)
        coordinate.pop(dimensao**2)
        
        peso[None] = 0
        coordinate[None] = (dimensao-1, dimensao-1)
        
        #posição do vazio
        self.vazio = (dimensao-1, dimensao-1)
        
        #tabuleiro
        self.tabuleiro = tabuleiro
        
        #dimensão do tabuleiro
        self.dimensao = dimensao
        
        self.peso = peso
        self.coordinate = coordinate
        
        #jogada anterior que gerou o tabuleiro atual
        self.jogada_anterior = jogada_anterior
    
    """
    Função para copiar um tabuleiro e mudar sua referencia
    """
    def copy(self):
        return copy.deepcopy(self)
    
    def mudaPeso(self, valor, pos, gabarito):
        pos_final = gabarito.coordinate[valor]
        valor_peso = abs(pos_final[0] - pos[0]) + abs(pos_final[1] - pos[1])
        self.peso[valor] = valor_peso
        
    """
    Função que simula uma jogada legal. Retorna outro tabuleiro com a jogada
    """
    def jogada(self, pos, gabarito):
        
        #presenvando o tabuleiro atual
        novo = self.copy()
        
        #pegando os vizinhos do vazio
        vizinhos = novo.getVizinhos()
        
        #se a jogada for um vizinho, a pos é valida!
        if pos in vizinhos:
            
            valor = novo.tabuleiro[pos[0]][pos[1]]
            
            novo.tabuleiro[self.vazio[0]][self.vazio[1]] = valor
            
            self.mudaPeso(valor, self.vazio, gabarito)  
            
            novo.tabuleiro[pos[0]][pos[1]] = None
            
            self.mudaPeso(None, pos, gabarito)
            
            novo.vazio = pos
            
            novo.jogada_anterior = pos
            
            novo.peso = self.peso
            
        return novo
    
    """
    Função que retorna as posições de jogadas valídas
    """
    def getVizinhos(self):
        
        #vendo os vizinhos do vazio
        pos = self.vazio
        possibilidades = [
            (pos[0]-1, pos[1]),
            (pos[0]+1, pos[1]),
            (pos[0], pos[1]+1),
            (pos[0], pos[1]-1)
        ]
        
        #tirando os que são ilegais
        possibilidades = filter(lambda x: (x[0] >= 0 and x[0] < self.dimensao) and 
               (x[1] >= 0 and x[1] < self.dimensao), possibilidades)
        
        return list(possibilidades)
    
    """
    Função que cria um tabuleiro randomico partindo de um gabarito
    """    
    def embaralha(self, vezes, gabarito):
        
        #presenvando o tabuleiro atual
        proximo = self.copy()
        
        #fazundo multiplas jogadas aleatorias
        for i in range(0,vezes):
            x = proximo.getVizinhos()
            
            pos = random.choice(x)
            
            proximo = proximo.jogada(pos, gabarito)
            
        return proximo
    
    """
    Função para verificar igualdade entre tabuleiros
    """
    def __eq__(self, outro):
        return (self.vazio == outro.vazio and self.tabuleiro == outro.tabuleiro)
    
    """
    Sobreescrita do print usando numpy
    """
    def __str__(self):
        return str(np.array(self.tabuleiro))
    

# O nó da árvore

Para montar a arvore, é necessário utilizar uma estrutura de nós com n filhos e 1 pai

In [3]:
class No:
    
    """
    Para construir o nó, é necessário passar seu conteúdo e seu pai que, por default, é None
    """
    def __init__(self, conteudo,pai = None):
        
        #pai do nó
        self.pai = pai
        
        #conteudo do nó
        self.conteudo = conteudo
        
        #lista de filhos
        self.filhos = list()
        
        self.pesoTotal = self.calculaPeso()
    
    def calculaPeso(self):
        # modo correto
        # peso_pai = 0 if not self.pai else self.pai.pesoTotal
        # modo errado
        peso_pai = 0
        return sum(self.conteudo.peso.values()) + peso_pai
    
    """
    Funçaõ para adicionar filhos ao nó, deve-se passar apenas o conteudo do filho
    """
    def add(self, conteudo):    
        filho = No(conteudo, pai = self)
        self.filhos.append(filho)
        
    """
    dois nós são iguais se se o conteúdo for o mesmo
    """
    def __eq__(self, outro):
        
        return (self.conteudo == outro.conteudo)
    
    def __lt__(self, other):
        return self.pesoTotal < other.pesoTotal

# Funções auxiliares

Para fazer o busca em largura de forma dinamica, foi necessário elaborar funções de verificação e manuntenção de arvore

In [4]:
"""
Função para verificar se, dado um nó, ele ja apareceu em sua linhagem
"""
def Correspondencia(no):
    if not no.pai:
        return False
    
    proximo = no.pai
    
    while proximo:
        
        if proximo == no:
            return True
        else:
            proximo = proximo.pai
    return False

In [5]:
"""
Função para, dado um nó, gerar seus filhos de jogadas lícitas
"""
def CriarFilhos(no, fim):
    t = no.conteudo
    
    pos = t.getVizinhos()
    
    filhos = []

    for coord in pos:
        
        j = t.jogada(coord, fim.conteudo)
        
        if not Correspondencia(No(j,pai = no)): filhos.append(No(j,pai = no))
    
    return filhos

# A Busca Em Largura Dinâmica

Finalmente, o algoritmo de busca. Ele gera uma arvore de possibilidades até achar uma resposta

In [6]:

"""
Algoritmo para achar o caminho de jogadas do jogo passado como parametro
"""
def BuscaEmLargura(problema, solucao):
    count = 0
    #se o seu problema é igual a solução...
    if problema == solucao:
        return problema
    
    #Gerarndo as primeiras possibilidades
    proximos = CriarFilhos(problema, solucao)
    
    #enquanto tiver possibilidades
    while len(proximos) != 0:
        count += 1
        filho = proximos.pop(0)
        
        #achei a solucao!
        if filho == solucao:
            print("Solved with ", count, "iterations")
            
            return filho
        
        for x in CriarFilhos(filho, solucao): proximos.append(x)
        
    print("Solved with", count, "iterations")
        
    return None  


def AEstrela(problema, solucao):
    count = 0
    #se o seu problema é igual a solução...
    if problema == solucao:
        return problema
    
    #Gerarndo as primeiras possibilidades
    proximos = CriarFilhos(problema, solucao)
    
    proximos.sort(key=lambda x: x.pesoTotal)
    
    #enquanto tiver possibilidades
    while len(proximos) != 0:
        count += 1
        filho = proximos.pop(0)
        
        #achei a solucao!
        if filho == solucao:
            print("Solved with ", count, "iterations")
            
            return filho
        
        for x in CriarFilhos(filho, solucao): bisect.insort(proximos, x)
        
        proximos.sort(key=lambda x: x.pesoTotal)
        
        if count%10000 == 0:
            print(count, "iterations", end=" ")
            input("deseja continuar? ")
        
    print("Solved with", count, "iterations")
        
    return None    

In [7]:
"""
Algoritmo para achar o conjunto de jogadas que originaram o nó
"""
def coletarJogadas(no):
    jogadas = []
    proximo = no
    while proximo.pai:
        jogadas.append(proximo.conteudo.jogada_anterior)
        proximo = proximo.pai
    return jogadas[::-1]        

# Demonstração

In [8]:
gabarito = Jogo(3)
problema = gabarito.embaralha(1000, gabarito)
print(problema)

[[None 3 1]
 [8 5 7]
 [2 4 6]]


In [9]:
inicio = No(problema)
fim = No(gabarito)
inicio.pesoTotal

18

In [ ]:
start_time = time.time()
resultado = BuscaEmLargura(inicio, fim) 
print("Execution time: %s" % time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))

In [10]:
start_time = time.time()
resultado = AEstrela(inicio, fim)
print("Execution time: %s" % time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))

Solved with  861 iterations
Execution time: 00:00:00


In [ ]:
for i in range()

In [ ]:
jogadas = coletarJogadas(resultado)
print(jogadas)